In [1]:
%load_ext autoreload
%autoreload 2

# 原始激活排行

In [2]:
import os
from ablation import *
from visualization import *
from ipywidgets import widgets

task_options = [name[5:-3] for name in os.listdir("rank_tensors")]

task_name1 = widgets.Dropdown(options=task_options, value=task_options[0], description='Task 1:', disabled=False)
task_name2 = widgets.Dropdown(options=task_options, value=task_options[1], description='Task 2:', disabled=False)

n_layer, n_neuron = GetLayerNeuron(task_name1.value)

layer_slider = widgets.IntSlider(value=1, min=0, max=n_layer-1, step=1, description='Layer:')

In [3]:
clusters = {}

from IPython.display import display

def update_plot(task_name1, task_name2, layer_index, horizonal, vertical):
    # 每次控件改变时，重新计算 cluster
    cluster = ClusterByRect(task_name1, task_name2, layer_index, horizonal, vertical)
    clusters[layer_index] = cluster
    PlotTwoRankingsWithRect(task_name1, task_name2, layer_index, horizonal, vertical, cluster)

horizonal = widgets.IntRangeSlider(value=[351, 1585], min=0, max=n_neuron, step=1, description='Horizonal Range:')
vertical = widgets.IntRangeSlider(value=[422, 3072], min=0, max=n_neuron, step=1, description='Vertical Range:')

ui = widgets.VBox([task_name1, task_name2, layer_slider, horizonal, vertical])
out = widgets.interactive_output(update_plot, {'task_name1': task_name1, 'task_name2': task_name2, 'layer_index': layer_slider, 'horizonal': horizonal, 'vertical': vertical})

display(ui, out)

Output()

In [4]:
clusters.keys()

dict_keys([1])

In [5]:
from subject_model import ModelSubject_GPT2, AblationSetting
from input_corpus import TextCorpusDispatcher
from tqdm.notebook import tqdm
import torch

model = ModelSubject_GPT2("gpt2")
dataset_name = "ag_news"
num_each = 500
batch_size = 1

batches_1, batches_2 = TextCorpusDispatcher(
    corpus_name=dataset_name,
    num_each=num_each,
    batch_size=batch_size,
)

difference = torch.zeros(model.n_vocab,)

for batch in tqdm(batches_1.batches, total=len(batches_1.batches)):
    probs_ablation_off = model.inference_ablation(
        inputs = batch.texts,
        ablations = []
    )
    probs_ablation_on = model.inference_ablation(
        inputs = batch.texts,
        ablations = [AblationSetting(
            layer=1,
            neurons=clusters[1]
        )]
    )
    for prob_off, prob_on in zip(probs_ablation_off, probs_ablation_on):
        difference += (prob_off - prob_on).sum(dim=0)

  0%|          | 0/500 [00:00<?, ?it/s]

In [7]:
# 看 difference 中哪些位置的值变动最大
topk = 10
topk_indices = torch.abs(difference).topk(topk).indices

topk_indices
difference

tensor([1.5402e+08, 1.5402e+08, 1.5402e+08,  ..., 1.5402e+08, 1.5402e+08,
        1.5402e+08], grad_fn=<AddBackward0>)